add volume thresholds and only trade when the underlying isn't going exponential against the spread positions Also only run a naive backtest

In [ ]:
from ib_insync import *
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# --- PARAMETERS ---
numbars = 14400
durationStr = '7 D'
barSizeSetting = '5 mins'
symbol1 = 'GOOG'
lastTradeDateOrContractMonth1 = '20250725'
strike1 = 180
right1 = 'P'    
symbol2 = 'GOOG'
lastTradeDateOrContractMonth2 = '20250725'
strike2 = 160
right2 = 'P'


# --- Connect to IBKR ---
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=np.random.randint(1000, 9999))

# --- Define option contracts ---
option1 = Option(symbol1, lastTradeDateOrContractMonth1, strike1, right1, 'SMART', 'USD')
option2 = Option(symbol2, lastTradeDateOrContractMonth2, strike2, right2, 'SMART', 'USD')
ib.qualifyContracts(option1, option2)

# --- Subscribe to live bars ---
bars1 = ib.reqHistoricalData(
    option1,
    endDateTime='',
    durationStr='2 D',
    barSizeSetting='5 mins',
    whatToShow='TRADES',
    useRTH=True,
    formatDate=1,
    keepUpToDate=True
)

bars2 = ib.reqHistoricalData(
    option2,
    endDateTime='',
    durationStr='2 D',
    barSizeSetting='5 mins',
    whatToShow='TRADES',
    useRTH=True,
    formatDate=1,
    keepUpToDate=True
)

# --- Strategy Handler with Plotly Chart ---
def on_new_bar(bars, has_new_bar):
    if not has_new_bar:
        return

    # Convert to DataFrames
    df1 = util.df(bars1).set_index('date').rename(columns={'close': 'Close1'})
    df2 = util.df(bars2).set_index('date').rename(columns={'close': 'Close2'})

    # Align and calculate spread
    datafix = df1.join(df2, how='inner')
    datafix['spread'] = datafix['Close1'] - datafix['Close2']

    # Bollinger Bands
    window = 20
    num_std = 2
    datafix['bb_middle'] = datafix['spread'].rolling(window).mean()
    datafix['bb_std'] = datafix['spread'].rolling(window).std()
    datafix['bb_upper'] = datafix['bb_middle'] + num_std * datafix['bb_std']
    datafix['bb_lower'] = datafix['bb_middle'] - num_std * datafix['bb_std']

    datafix['prev_spread'] = datafix['spread'].shift(1)
    datafix['prev_bb_lower'] = datafix['bb_lower'].shift(1)
    datafix['prev_bb_upper'] = datafix['bb_upper'].shift(1)

    datafix['bb_signal'] = 'hold'
    buy_mask = (datafix['prev_spread'] > datafix['prev_bb_lower']) & (datafix['spread'] < datafix['bb_lower'])
    sell_mask = (datafix['prev_spread'] < datafix['prev_bb_upper']) & (datafix['spread'] > datafix['bb_upper'])
    datafix.loc[buy_mask, 'bb_signal'] = 'buy'
    datafix.loc[sell_mask, 'bb_signal'] = 'sell'

    # Get latest signal
    last_row = datafix.iloc[-1]
    signal = last_row['bb_signal']
    print(f"New bar at {last_row.name} | Signal: {signal}")

    # Check current positions
    positions = {p.contract.conId: p.position for p in ib.positions()}
    pos1 = positions.get(option1.conId, 0)
    pos2 = positions.get(option2.conId, 0)

    if pos1 == 1 and pos2 == -1:
        current_position = 'long'
    elif pos1 == -1 and pos2 == 1:
        current_position = 'short'
    else:
        current_position = 'flat'

    # Trading logic
    if current_position == 'long' and signal == 'sell':
        print("Exiting LONG spread...")
        ib.placeOrder(option1, MarketOrder('SELL', 1))
        ib.placeOrder(option2, MarketOrder('BUY', 1))

    elif current_position == 'short' and signal == 'buy':
        print("Exiting SHORT spread...")
        ib.placeOrder(option1, MarketOrder('BUY', 1))
        ib.placeOrder(option2, MarketOrder('SELL', 1))

    elif current_position == 'flat':
        if signal == 'buy':
            print("Entering LONG spread...")
            ib.placeOrder(option1, MarketOrder('BUY', 1))
            ib.placeOrder(option2, MarketOrder('SELL', 1))
        elif signal == 'sell':
            print("Entering SHORT spread...")
            ib.placeOrder(option1, MarketOrder('SELL', 1))
            ib.placeOrder(option2, MarketOrder('BUY', 1))
        else:
            print("Holding position.")

    # --- Plotting ---
    plot_df = datafix.tail(50)  # Show last 50 bars
    buy_signals = plot_df[plot_df['bb_signal'] == 'buy']
    sell_signals = plot_df[plot_df['bb_signal'] == 'sell']

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['spread'], name='Spread', line=dict(color='black')))
    fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['bb_upper'], name='BB Upper', line=dict(color='green', dash='dash')))
    fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['bb_lower'], name='BB Lower', line=dict(color='red', dash='dash')))
    fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['bb_middle'], name='BB Middle', line=dict(color='orange', dash='dot')))
    fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['spread'],
                             mode='markers', name='Buy Signal',
                             marker=dict(symbol='triangle-up', color='green', size=12)))
    fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['spread'],
                             mode='markers', name='Sell Signal',
                             marker=dict(symbol='triangle-down', color='red', size=12)))

    fig.update_layout(
        title='Live Spread with Bollinger Bands and Signals',
        xaxis_title='Time',
        yaxis_title='Spread',
        template='plotly_white',
        width=1000,
        height=500
    )
    fig.show()

# --- Attach the handler to bar updates ---
bars1.updateEvent += on_new_bar
bars2.updateEvent += on_new_bar

print("Running strategy with live chart... Press Ctrl+C to stop.")
ib.run()
